In [1]:
import pandas as pd
from utils import load_filtered_data

In [2]:
city = 'Sicklerville'
name = 'mixed'

# load city data
data = load_filtered_data(city)

In [3]:
# creating the nodes

nodes = data['user'].drop(columns=['friends'])
nodes = nodes.rename(columns={'user_id': 'id', 'name': 'people_name'})

nodes

,id,people_name,review_count,yelping_since,useful,funny,cool,elite,fans,average_stars,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,RgDVC3ZUBqpEe6Y1kPhIpw,Monica,1282,2009-02-23 14:08:31,12640,10005,11149,"2009,2010,2011,2012,2013,2014,2015,2016",804,4.17,...,102,108,138,65,898,2584,1731,1731,551,177
1,zkamNMEjihh3zN7lC7_WVw,Matthew,15,2006-02-04 15:53:46,35,4,22,NaN,1,4.07,...,1,0,0,0,1,3,2,2,0,0
2,gVFxZMcuG_Tal2_TnpmUPg,Stephanie,146,2008-05-05 00:16:33,347,100,124,"2011,2012,2013,2014,2015,2016",6,3.68,...,13,1,0,0,10,8,21,21,8,0
3,g0_x4kVvJAYuk96oCcbOmw,Michael,455,2009-04-15 12:46:06,1823,696,663,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2018",46,3.38,...,11,0,1,5,39,33,57,57,32,8
4,-NbeVN5tnwdyYAvdNkKMjw,Dominic,1101,2012-04-25 19:31:00,6704,1959,4179,"2012,2013,2014,2015,2016,2017,2018,2019,20,20,...",193,3.75,...,33,18,2,2,312,457,637,637,137,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073,CUE13op55l9WZp6fBTUWnA,Nicole,2,2016-08-19 13:24:47,0,0,0,NaN,0,1.00,...,0,0,0,0,0,0,0,0,0,0
3074,cACn6Qc1owO7hV_ZhcP74Q,Allison,1,2019-11-10 00:05:23,0,0,0,NaN,0,5.00,...,0,0,0,0,0,0,0,0,0,0
3075,SpTYEhfInvALG1le1uu21w,Ray,2,2016-01-18 16:55:04,5,0,0,NaN,0,1.00,...,0,0,0,0,0,0,0,0,0,0
3076,sGOCaCP7_SMlaRKPNO__CQ,Tahjae,2,2019-05-04 13:07:23,0,0,0,NaN,0,5.00,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# creating the edges for users&bgt

temp_edges_struct = {'from': [], 'to': [], 'weight': [], 'type': []}
connected = {}

for business in data['review']['business_id'].unique():
    reviews_of_business = data['review'][data['review']['business_id'] == business]
    users_that_reviewed_business = reviews_of_business['user_id'].unique()
    for i in range(len(users_that_reviewed_business)-1):
        for j in range(i+1, len(users_that_reviewed_business)):
            if users_that_reviewed_business[i] in connected:
                connected[users_that_reviewed_business[i]].append(users_that_reviewed_business[j])
            else:
                connected[users_that_reviewed_business[i]] = [users_that_reviewed_business[j]]
            if users_that_reviewed_business[j] in connected:
                connected[users_that_reviewed_business[j]].append(users_that_reviewed_business[i])
            else:
                connected[users_that_reviewed_business[j]] = [users_that_reviewed_business[i]]

for business in data['review']['business_id'].unique():
    reviews_of_business = data['review'][data['review']['business_id'] == business]
    users_that_reviewed_business = reviews_of_business['user_id'].unique()
    for i in range(len(users_that_reviewed_business)-1):
        for j in range(i+1, len(users_that_reviewed_business)):
            # from i to j
            temp_edges_struct['from'].append(users_that_reviewed_business[i])
            temp_edges_struct['to'].append(users_that_reviewed_business[j])
            temp_edges_struct['weight'].append(connected[users_that_reviewed_business[i]].count(users_that_reviewed_business[j]))
            temp_edges_struct['type'].append('reviewed-same-restaurant')
            
            # from j to i
            temp_edges_struct['from'].append(users_that_reviewed_business[j])
            temp_edges_struct['to'].append(users_that_reviewed_business[i])
            temp_edges_struct['weight'].append(connected[users_that_reviewed_business[j]].count(users_that_reviewed_business[i]))
            temp_edges_struct['type'].append('reviewed-same-restaurant')

edges_usersbgt = pd.DataFrame(temp_edges_struct)

# remove repeated lines
edges_usersbgt = edges_usersbgt.drop_duplicates()

# adding the edges for friendships

user_ids_set = set(data['user']['user_id'])
temp_edges_struct = {'from': [], 'to': [], 'weight': [], 'type': []}

for index, row in data['user'].iterrows():
    user_id = row['user_id']
    friends = row['friends']
    if pd.notnull(friends):
        friend_list = friends.split(', ')
        for friend in friend_list:
            friendship_pair = tuple(sorted([user_id, friend]))
            if friendship_pair[0] in user_ids_set and friendship_pair[1] in user_ids_set:
                temp_edges_struct['from'].append(friendship_pair[0])
                temp_edges_struct['to'].append(friendship_pair[1])
                temp_edges_struct['weight'].append(10)
                temp_edges_struct['type'].append('friendship')

edges_friendships = pd.DataFrame(temp_edges_struct)
edges_friendships = edges_friendships.drop_duplicates()

edges = pd.concat([edges_usersbgt, edges_friendships])

def multiply_values(x):
    result = 1
    for value in x:
        result *= value
    return result

edges = edges.groupby(['from', 'to']).agg({'weight': multiply_values}).reset_index()

edges = edges.drop_duplicates()

edges['type'] = 'reviewed-same-restaurant-and-friendship'

edges

,from,to,weight,type
0,-0MIp6WKJ8QvGnYZQ5ETyg,-ZCf13dTuzGKMwKFRxTUwQ,1,reviewed-same-restaurant-and-friendship
1,-0MIp6WKJ8QvGnYZQ5ETyg,-u5CsCUHOvP5OHYaNGteZw,1,reviewed-same-restaurant-and-friendship
2,-0MIp6WKJ8QvGnYZQ5ETyg,0cLvdT2ANqM7CL93gccVjA,1,reviewed-same-restaurant-and-friendship
3,-0MIp6WKJ8QvGnYZQ5ETyg,2K6e4rSuEYKJpmU37Kk-oA,1,reviewed-same-restaurant-and-friendship
4,-0MIp6WKJ8QvGnYZQ5ETyg,2RCnyrzyOQ1oA9jSgL7XPQ,1,reviewed-same-restaurant-and-friendship
...,...,...,...,...
275340,zyTCJLh4oUsuRar4kxRfgg,vnSNGuRxa9p3Xrvlj3t03g,1,reviewed-same-restaurant-and-friendship
275341,zyTCJLh4oUsuRar4kxRfgg,w7QCKFrjpaBHiPSOo2O3MQ,1,reviewed-same-restaurant-and-friendship
275342,zyTCJLh4oUsuRar4kxRfgg,xvdkeWY0421VZicMbYmz2w,1,reviewed-same-restaurant-and-friendship
275343,zyTCJLh4oUsuRar4kxRfgg,y2PUG5elRIz6xkQP30DScA,1,reviewed-same-restaurant-and-friendship


In [9]:
# save data
nodes.to_csv(f'nodes_and_edges/{city}_{name}_nodes.csv', index=False)
edges.to_csv(f'nodes_and_edges/{city}_{name}_edges.csv', index=False)